In [ ]:
# define instances

import os

class ParameterSet:
  def __init__(self, name: str, variants: list):
    self.name = name
    self.variants = variants

  def expand(self)-> list[str]:
    return []

class EnvVarSet(ParameterSet):
  def __init__(self, name: str, variants: list):
    super().__init__(name, variants)

# num_block_param = EnvVarSet("NUM_BLOCKS", [1,2,4,8,16,32])
num_block_param = EnvVarSet("NUM_BLOCKS", [32])
matrix_size_param = EnvVarSet("MATRIX_SIZE", [2048, 4096, 6144,  8192, 10240, 12288, 14336, 16384])
# matrix_size_param = EnvVarSet("MATRIX_SIZE", [2048])

# num_block_param = EnvVarSet("NUM_BLOCKS", [4])
# matrix_size_param = EnvVarSet("MATRIX_SIZE", [8])

# scheduler_mode_param = EnvVarSet("SCHEDULER_MODE", ["virtual-fifo"])omp-task
# scheduler_mode_param = EnvVarSet("SCHEDULER_MODE", ["omp-task","virtual-fifo"])
scheduler_mode_param = EnvVarSet("SCHEDULER_MODE", ["omp-task", "sequential", "virtual-fifo"])

class Experiment:
  def __init__(self, name: str):
    self.name = name

  def build(self):

    experiment_dir = f"{os.environ["IARA_DIR"]}/experiment/cholesky"
    sources_path = f"{os.environ["IARA_DIR"]}/test/Iara/05-cholesky"

    %cd {experiment_dir}
    ! mkdir -p instances
    %cd instances


    # Generate all possible combinations of parameters

    param_sets = [matrix_size_param, num_block_param, scheduler_mode_param]
    instance_params=[((param_sets[-1], i),) for i in param_sets[-1].variants]
    param_sets.pop()
    while len(param_sets) > 0:
      this_params = [(param_sets[-1], i) for i in param_sets[-1].variants]
      new_params = []
      for i in this_params:
        for j in instance_params:
          new_params.append((i, *j))
      instance_params = new_params
      param_sets.pop(-1)

    for i in instance_params:

      %cd {experiment_dir}/instances

      instance_name = f"{i[0][1]}_{i[1][1]}_{i[2][1]}"

      ! mkdir -p {instance_name}
      %cd {instance_name}

      instance_dir = ! realpath .
      print(f"{instance_dir=}")
      instance_dir = instance_dir[0]

      ! mkdir -p build

      command = " ".join(f"{j.name}={k}" for j, k in i)

      build_path = ! realpath build

      print(f"{build_path=}")

      # command += f" PATH_TO_TEST_BUILD_DIR={instance_dir} EXTRA_RUNTIME_ARGS=-DIARA_DEBUGPRINT build-single-test.sh {sources_path}"
      command += f" PATH_TO_TEST_BUILD_DIR={instance_dir} build-single-test.sh {sources_path}"

      print(command)

      %cd {sources_path}

      ! {command}




cholesky_experiment = Experiment("cholesky")

cholesky_experiment.build()

